<a href="https://colab.research.google.com/github/edyadan/simple/blob/master/googledrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
download_by_folder('08_04_2021','/home','.pdf')

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'mi-proyecto-304014'
bucket_name = 'cloud-ai-platform-011fa2a5-83d5-4678-b9ea-4367383a3a10'
!gcloud config set project {project_id}
!gsutil cp gs://{bucket_name}/credentials.json /content
!gsutil cp gs://{bucket_name}/mime.json /content
!gsutil cp gs://{bucket_name}/storage.json /content
!pip install pdf2image
!apt-get install poppler-utils

In [ ]:
from google.colab import files
 
uploaded = files.upload()
 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [6]:
from apiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from apiclient.http import MediaIoBaseDownload
from apiclient.http import MediaFileUpload
import pandas as pd
import io
import json
from pdf2image import convert_from_path
 
SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = discovery.build('drive', 'v2', http=creds.authorize(Http()))
def upload(file_name,path_dest):
    response_folder = service.files().list(q="mimeType='application/vnd.google-apps.folder'",
                                      spaces='drive',
                                      fields="items(id,title)",
                                      pageToken=None).execute()
    df = pd.DataFrame(response_folder['items'])
    idfolder = df[ (df['title'] == path_dest)]['id'][0]
    ext = file_name.split('.')[-1]
    with open('mime.json') as json_file:
        data = json.load(json_file)
    df = pd.DataFrame(data['items'])
    mimetype = df[ (df['ext'] == ext)]['mime'].values[0]    
    name = file_name.split('/')[-1]
    body = {
    'title': name,
    'mimeType': mimetype,
    "parents": [{"id": idfolder, "kind": "drive#childList"}] }                 
    media_body = MediaFileUpload(file_name, mimetype=mimetype, resumable=True)
    res = service.files().insert(
                body=body,
                media_body= media_body).execute()
    return True
def download_by_folder(folder_orig, path_dest, file_name):
    response_folder = service.files().list(q="mimeType='application/vnd.google-apps.folder'",
                                      spaces='drive',
                                      fields="items(id,title)",
                                      pageToken=None).execute()
    df = pd.DataFrame(response_folder['items'])
    idfolder = df[ (df['title'] == folder_orig)]['id'][0]
    
    query = "'{}' in parents and title contains '{}'".format(idfolder, file_name)
    response_files = service.files().list(q=query,
                                      spaces='drive',
                                      fields="items(id,title)",
                                      pageToken=None).execute()
    for k in response_files['items']:
        tmp_file = path_dest + '/' + k['title']
        fh = io.FileIO(tmp_file,'wb')     
        request1 = service.files().get_media(fileId=k['id'])
        downloader = MediaIoBaseDownload(fh, request1)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        fh.close()
        '''convert'''
        pages = convert_from_path(tmp_file, 300)
        pages[0].save(tmp_file.split('.')[0]+'.jpg','JPEG')
        upload(tmp_file.split('.')[0]+'.jpg',folder_orig)
        print(tmp_file)
def id_files_subfolder(folder,subfolder,file_name):
    response_folder = service.files().list(q="mimeType='application/vnd.google-apps.folder'",
                                      spaces='drive',
                                      fields="items(id,title,parents)",
                                      pageToken=None).execute()
    df = pd.DataFrame(response_folder['items'])
    df['parents']=df['parents'].apply(lambda x : x[0]['id'] if x else '' )
    idfolder = df[ (df['title'] == folder)]['id'].values[0]
    idsubfolder =  df[ (df['parents'] == idfolder)]['id'].values[0]
    query = "'{}' in parents and title contains '{}'".format(idsubfolder, file_name)
    response_files = service.files().list(q=query,
                                      spaces='drive',
                                      fields="items(id,title)",
                                      pageToken=None).execute()
    idfile=response_files['items'][0]['id']
    return idfile         
def download_file(path_dest,file_name):
    query = "title contains '{}'".format(file_name)
    response_files = service.files().list(q=query,
                                      spaces='drive',
                                      fields="items(id,title)",
                                      pageToken=None).execute()
    for k in response_files['items']:
        tmp_file = tmp_file = '/' + path_dest + '/' + k['title']
        fh = io.FileIO(tmp_file,'wb')     
        request1 = service.files().get_media(fileId=k['id'])
        downloader = MediaIoBaseDownload(fh, request1)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        fh.close()
        return file_name

In [9]:
!ls /content

adc.json  credentials.json  mime.json  sample_data  storage.json  tmp
